In [3]:
# Imports
import PySimpleGUI as sg
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os
import scipy
import numpy as np
import math

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
file_path = os.path.join(parent_directory, "model", "monkey_data.mat")
other_fp = os.path.join(parent_directory, "model", "hardcore_only_loss_model_outputs.pickle")

# Load the .mat file
mat_data = scipy.io.loadmat(file_path)
inp_data = mat_data['inp']
print(inp_data.shape)
for ind,trash in enumerate(inp_data[0]):
    inp_data[0][ind] = inp_data[0][ind][20]
hold_data = inp_data

mvm_data = mat_data['targ']
print(mvm_data.shape)
print(mvm_data[0][0].shape)
print(mvm_data[0][1:200])


first = np.random.choice(mvm_data[0].shape[0], 200, replace=False)  
second = np.random.choice(mvm_data[0].shape[0], 200, replace=False) 

(1, 502)
(1, 502)
(50, 298)
[array([[ 0.        ,  0.        ,  0.        , ...,  0.00745155,
          0.01288072,  0.01680898],
        [ 0.        ,  0.        ,  0.        , ...,  0.08523967,
          0.06180861,  0.04126479],
        [ 0.        ,  0.        ,  0.        , ...,  0.00956578,
          0.00586271,  0.00302448],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.37760645,
         -0.34920594, -0.32296256],
        [ 0.        ,  0.        ,  0.        , ..., -0.17194301,
         -0.17086133, -0.16869355],
        [ 0.        ,  0.        ,  0.        , ..., -0.20003636,
         -0.19653433, -0.1907099 ]])
 array([[ 0.        ,  0.        ,  0.        , ...,  0.01512346,
          0.01981549,  0.0228777 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.08172533,
          0.06504852,  0.05032363],
        [ 0.        ,  0.        ,  0.        , ...,  0.00899092,
          0.0049552 ,  0.00193927],
        ...,
        [ 0.        ,  0.  

In [5]:
# Akinesia : want some sort of metric for how long movement initialization is delayed
# Returns the average delay over trial per muscle from 'test' data compared to 'base' data, 
#       using the hold signals as specified in input
#       Also returns some measure of its statistical significance
#
#       input is shape [numtrials][1][length of each trial]
#       test and base should have shape [numtrials][50][length of each trial]

def akinesia(test, base, input_test, input_base):

    # loop over the test trials creating all_test_delays, a trials x 50 matrix with the delay from each muscle during each trial
    num_trials_test = input_test.shape[0]
    all_test_delays = np.zeros((num_trials_test, 50))
    for ind,trial in enumerate(input_test):
        inxs = np.argwhere(trial<1)
        go_cue = inxs[0]
        
        # 50 x 200 smth trial
        curr_test_trial = test[ind]
        muscle_mvmts_test = np.zeros((50))
        for i, muscle in enumerate(curr_test_trial):
            # for each mucle, what is the first time that it is non-zero
            mvms = np.argwhere(muscle!=0)
            if mvms.size != 0:
                muscle_mvmts_test[i] = mvms[0]
        # subtract that index from the go_cue !! NEEDS TO BE CHANGED TO GO CUE FROM IMAGE !!
        all_test_delays[ind] = muscle_mvmts_test - go_cue

    # same thing as above for base trials
    num_trials_base = input_base.shape[0]
    all_base_delays = np.zeros((num_trials_base, 50))
    for ind,trial in enumerate(input_base):
        inxs = np.argwhere(trial<1)
        go_cue = inxs[0]

        # 50 x 200 smth trial
        curr_base_trial = base[ind]
        muscle_mvmts_base = np.zeros((50))
        for i, muscle in enumerate(curr_base_trial):
            mvms = np.argwhere(muscle!=0)
            if mvms.size != 0:
                muscle_mvmts_base[i] = mvms[0]
       

        all_base_delays[ind] = muscle_mvmts_base - go_cue
    
    # takes the average of the delays over the trials axis for both test and base, 
    # then subtracts the difference for a comparitive index
    # 1 x 50
    avg_delay = np.average(all_test_delays, axis = 0) - np.average(all_base_delays, axis = 0)

    # 2-sample t-score --> p-value test
    # What's the probability that these samples came from the same distribution?
    mean_dist_base = np.average(all_base_delays, axis = 0)
    std_dist_base = scipy.stats.tstd(all_base_delays, axis = 0)
    mean_dist_test = np.average(all_test_delays, axis = 0)
    std_dist_test = scipy.stats.tstd(all_test_delays, axis = 0)

    t_score = (mean_dist_base - mean_dist_test) / np.sqrt((std_dist_base**2 / num_trials_base) + (std_dist_test**2 / num_trials_test))
    df = num_trials_base + num_trials_test - 2
    p_value = 2 * (1 - scipy.stats.t.cdf(np.abs(t_score), df))
    
    return avg_delay, p_value

        


In [6]:
akinesia(mvm_data[0][first], mvm_data[0][second], hold_data[0][first], hold_data[0][second])

/var/folders/q8/wlhng4j113sg8_tllhtdj7_m0000gn/T/ipykernel_37983/3298697781.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  muscle_mvmts_test[i] = mvms[0]
/var/folders/q8/wlhng4j113sg8_tllhtdj7_m0000gn/T/ipykernel_37983/3298697781.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  muscle_mvmts_base[i] = mvms[0]
/var/folders/q8/wlhng4j113sg8_tllhtdj7_m0000gn/T/ipykernel_37983/3298697781.py:59: RuntimeWarning: invalid value encountered in divide
  t_score = (mean_dist_base - mean_dist_test) / np.sqrt((std_dist_base**2 / num_trials_base) + (std_dist_test**2 / num_trials_test))


(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]))

In [7]:
# Hypokinesia : movement amplitude, take the integral under the velocity for ‘distance traveled’
#       Direct comparison of the overall movement amplitude (sum not weighted) between two samples
#       first each sample set is looped over and the sum of the velocities for each trial is added to 
#           an array. (summed over time, so the result is 50 x 1)
#       then, the average is taken over each trial (result is 50 x 1)
#       We return the difference between the averages for test and base (50 x 1)
#       2 sample p value is calculated, 

#       input arrays are [num_trials] [50 x length of trial]
#`      num trials for test and base can be different

# Returns:
#       average distance for both base and test, and the probability that both samples were taken from the same distribution

def hypokinesia(test, base):

    # loop through all the trials in 'base', sum up the velocities across all muscles, still keeping them seperate
    # store in all_trial_int_base a trials x 50 matrix
    trials_base = base
    num_trials_base = base.shape[0]
    all_trial_int_base = np.zeros((num_trials_base, 50))
    
    for ind,trial in enumerate(trials_base):
        all_trial_int_base[ind] = np.sum(trial, axis = 1)
    
    # same thing as above but for 'test' 
    trials_test = test
    num_trials_test = test.shape[0]
    all_trial_int_test = np.zeros((num_trials_test, 50))
    for ind,trial in enumerate(trials_test):
        all_trial_int_test[ind] = np.sum(trial, axis = 1)
    
    # average both across trials, 1 x 50
    avg_dist_base = np.average(all_trial_int_base, axis = 0)
    avg_dist_test = np.average(all_trial_int_test, axis = 0)

    # 2-sample t-score --> p-value test
    # What's the probability that these samples came from the same distribution?
    mean_dist_base = np.sum(avg_dist_base)
    std_dist_base = scipy.stats.tstd(np.sum(all_trial_int_base, axis = 1))
    mean_dist_test = np.sum(avg_dist_test)
    std_dist_test = scipy.stats.tstd(np.sum(all_trial_int_test, axis = 1))

    t_score = (mean_dist_base - mean_dist_test) / np.sqrt((std_dist_base**2 / num_trials_base) + (std_dist_test**2 / num_trials_test))
    df = num_trials_base + num_trials_test - 2
    p_value = 2 * (1 - scipy.stats.t.cdf(np.abs(t_score), df))
    
    return avg_dist_test, avg_dist_base, p_value
    


In [8]:
 
hypokinesia(mvm_data[0][first], mvm_data[0][second])

(array([-20.19755865, -11.83416584,  21.96248549,  -2.28699915,
         19.72769684, -17.52421071,  16.29071667,  16.19977046,
        -19.86798071, -19.74378763, -19.35263811,  17.51324503,
         18.08221712,  17.60890325, -20.40959855,  12.96367265,
        -13.34542769, -13.34542655, -13.35626573,  -4.50932334,
         -5.94013352,  -6.80515373,  14.41052577,  15.01426369,
         12.82712441,  -4.96183186,  -9.45105094,   9.31763855,
         10.10428818,   9.31259914,   7.29271249,   5.0183969 ,
          8.32513965,   7.06755814,   2.5746224 ,  -2.11759765,
         -6.03927405,  -7.53608781,  -5.29451728,  -6.49734826,
         -1.39060689,  -2.88015976,  -4.68661683,   4.55481624,
         -6.12960154,   4.53604439,   0.49590991,   6.32373679,
          7.82195066,   7.49406036]),
 array([-20.22127167, -11.52008467,  21.95722295,  -1.70278408,
         19.71757949, -17.29233513,  16.08306412,  16.06463367,
        -20.04873574, -19.9641412 , -19.66100922,  17.48159297,
  

In [9]:
print(mvm_data[0][second].shape)

(200,)


In [10]:
# Bradykinesia: lower curve or hesitation in the velocity
#       lower curve can be tested by comparing the maximum velocities
#       hesitation, not sure, might have to do with oscillations in the velocity 
#           could also test how quickly they get to max velocity?


def bradykinesia(test, base):

    # loop through the trails for 'base', taking the maximum velocity and storing it
    # in all_trial_max_base, trials x 50
    num_trials_base = base.shape[0]
    all_trial_max_base = np.zeros((num_trials_base, 50))
    
    for ind,trial in enumerate(base):
        all_trial_max_base[ind] = np.max(trial, axis = 1)

    # same thing as above but for 'test'
    num_trials_test = test.shape[0]
    all_trial_max_test = np.zeros((num_trials_test, 50))
    
    for ind,trial in enumerate(test):
        all_trial_max_test[ind] = np.max(trial, axis = 1)

    # take the average over all trials, 1 x 50
    avg_max_base = np.average(all_trial_max_base, axis = 0)
    avg_max_test = np.average(all_trial_max_test, axis = 0)    

    std_dist_base = scipy.stats.tstd(all_trial_max_base, axis = 0)
    std_dist_test = scipy.stats.tstd(all_trial_max_test, axis = 0)
    
     # 2-sample t-score --> p-value test
    # What's the probability that these samples came from the same distribution?
    t_score = (avg_max_base - avg_max_test) / np.sqrt((std_dist_base**2 / num_trials_base) + (std_dist_test**2 / num_trials_test))
    df = num_trials_base + num_trials_test - 2
    p_values = 2 * (1 - scipy.stats.t.cdf(np.abs(t_score), df))

    return avg_max_base, avg_max_test, p_values

In [11]:
bradykinesia(mvm_data[0][first], mvm_data[0][second])

(array([0.01419668, 0.18313803, 0.94203916, 0.4847581 , 0.80394554,
        0.12500301, 0.85758693, 0.79762381, 0.01483418, 0.01662789,
        0.02063639, 0.82490567, 0.83643313, 0.81932776, 0.01576868,
        0.77932125, 0.04923468, 0.04923467, 0.04953511, 0.04335845,
        0.15299174, 0.1233398 , 0.87323278, 0.90227405, 0.74070078,
        0.09642084, 0.19257874, 0.59362263, 0.65572324, 0.59570455,
        0.43558278, 0.31747802, 0.63932415, 0.60110715, 0.3480157 ,
        0.3391107 , 0.17969497, 0.1266735 , 0.17425583, 0.14683316,
        0.26490207, 0.18866319, 0.10026447, 0.40552534, 0.15363593,
        0.4332586 , 0.26802825, 0.53868949, 0.49779326, 0.44204835]),
 array([0.01411537, 0.17688879, 0.94384488, 0.47063847, 0.80082362,
        0.1152683 , 0.86543508, 0.80310384, 0.01209816, 0.01357264,
        0.01702113, 0.8280152 , 0.83931384, 0.82152946, 0.0125499 ,
        0.78298019, 0.04897382, 0.04897382, 0.0492812 , 0.04448766,
        0.15410257, 0.13297475, 0.87208403, 0.

In [12]:
def vel_to_acc(data):
    accs = []
    
    for trial in enumerate(data):
        # print(trial[1])
        accs.append(np.diff(trial[1]))
    
    return accs

In [23]:
# Tremor Factor: It is defined as the root mean square value of the acceleration of the arm during the reaching task. 
# this actually isn't finished I don't think...

def tremor_factor(test, base):
    base_acc = vel_to_acc(base)
    num_base_trials = len(base_acc)
    
    hold_arr = np.zeros((num_base_trials, 50))
    #Calculate square
    for i,trial in enumerate(base_acc):
        for j,muscle in enumerate(trial):
            square = 0
            mean = 0.0
            root = 0.0
            for time_point in muscle:
                square += time_point**2
     
            #Calculate Mean 
            mean = (square / (float)(muscle.shape[0]))
     
            #Calculate Root
            root = math.sqrt(mean)
            hold_arr[i][j] = root
    
    avgd_base = np.average(hold_arr, axis = 0)

    test_acc = vel_to_acc(test)
    num_test_trials = len(test_acc)
    
    hold_arr_test = np.zeros((num_test_trials, 50))
    #Calculate square
    for i,trial in enumerate(test_acc):
        for j,muscle in enumerate(trial):
            square = 0
            mean = 0.0
            root = 0.0
            for time_point in muscle:
                square += time_point**2
     
            #Calculate Mean 
            mean = (square / (float)(muscle.shape[0]))
     
            #Calculate Root
            root = math.sqrt(mean)
            hold_arr_test[i][j] = root
    
    avgd_test = np.average(hold_arr_test, axis = 0) 

    std_base = scipy.stats.tstd(hold_arr, axis = 0)
    std_test = scipy.stats.tstd(hold_arr_test, axis = 0)
    
     # 2-sample t-score --> p-value test
    # What's the probability that these samples came from the same distribution?
    t_score = (avgd_base - avgd_test) / np.sqrt((std_base**2 / num_base_trials) + (std_test**2 / num_test_trials))
    df = num_base_trials + num_test_trials - 2
    p_values = 2 * (1 - scipy.stats.t.cdf(np.abs(t_score), df))
      
    return avgd_base, avgd_test, p_values

In [24]:
tremor_factor(mvm_data[0][first], mvm_data[0][second])

(array([0.01663343, 0.01757525, 0.01766114, 0.01987299, 0.01505181,
        0.01978519, 0.01883215, 0.01607328, 0.0132484 , 0.01251872,
        0.01126179, 0.01593091, 0.01595983, 0.01546812, 0.01343293,
        0.01722471, 0.0178002 , 0.0178002 , 0.01785119, 0.00715579,
        0.01229781, 0.00981434, 0.01882926, 0.01929912, 0.01579272,
        0.01029858, 0.01359001, 0.01323802, 0.01486669, 0.01337199,
        0.00924259, 0.00705317, 0.01551285, 0.01507321, 0.00998156,
        0.01675212, 0.0119898 , 0.01189035, 0.01177859, 0.01384132,
        0.01167701, 0.01039465, 0.00944925, 0.01316547, 0.01194567,
        0.01403612, 0.01390282, 0.02038599, 0.01187375, 0.0120643 ]),
 array([0.01664443, 0.01765137, 0.01770605, 0.01996256, 0.01493676,
        0.01969645, 0.01893417, 0.01618796, 0.01310175, 0.01234613,
        0.01102533, 0.01599962, 0.01602815, 0.01553346, 0.0132864 ,
        0.01720562, 0.01775257, 0.01775257, 0.01780347, 0.00707406,
        0.0123004 , 0.00983772, 0.01879429, 0.